In [ ]:
from pasteur.kedro.ipython import * # type: ignore
register_kedro() # type: ignore

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84


In [ ]:
table = catalog.load("tab_texas_charges.wrk.idx_table")
trn = catalog.load("tab_texas_charges.trn.table")

In [ ]:
attr = trn["idx"].get_attributes()

In [ ]:
table.nunique()


revenue_code              461
hcpcs_qualifier             6
hcpcs_procedure_code     5146
modifier_1                155
modifier_2                 48
modifier_3                  4
modifier_4                  5
unit_measurement_code       8
units_of_service            8
unit_rate                  13
chrgs_line_item            10
chrgs_non_cov              17
dtype: int64


In [ ]:
%load_ext line_profiler

In [ ]:
# Oprimise rebalancer
from pasteur.hierarchy import rebalance_value, make_grouping, find_smallest_group, create_node_to_group_map

In [ ]:
target = "hcpcs_procedure_code"
val = attr[target][target]
# val

In [ ]:
rebalance_value(table[target], val)

In [ ]:
%lprun -f make_grouping rebalance_value(table[target], val)

Timer unit: 1e-06 s

Total time: 1.08502 s
File: /mnt/ext/projects/pasteur/src/pasteur/hierarchy.py
Function: make_grouping at line 224

Line #      Hits         Time  Per Hit   % Time  Line Contents
   224                                           def make_grouping(counts: np.ndarray, head: Level, common: int = 0) -> np.ndarray:
   225                                               """Converts the hierarchical attribute level tree provided into a node-to-group
   226                                               mapping, where `group[i][j] = z`, where `i` is the height of the mapping
   227                                               `j` is node `j` and `z` is the group the node is associated at that height.
   228                                           
   229                                               `counts` provides the class densities. It doesn't need to be normalized and
   230                                               some of its values may be *negative*.
   231    

In [ ]:
%lprun -f create_node_to_group_map rebalance_value(table[target], val)

Timer unit: 1e-06 s

Total time: 0.332929 s
File: /mnt/ext/projects/pasteur/src/pasteur/hierarchy.py
Function: create_node_to_group_map at line 200

Line #      Hits         Time  Per Hit   % Time  Line Contents
   200                                           def create_node_to_group_map(tree: list, grouping: np.ndarray, ofs: int = 0):
   201                                               """Traverses `tree` and creates an array which maps nodes to groups such that:
   202                                           
   203                                               `arr[x] = y`, where `x` is the node and `y` is its group.
   204                                           
   205                                               `grouping` is updated in place to form arr recursively and `ofs` is
   206                                               used to keep track of the current group number during recursion."""
   207                                           
   208    108328      3122

In [ ]:
%lprun -f find_smallest_group rebalance_value(table[target], val)

Timer unit: 1e-06 s

Total time: 0.993943 s
File: /mnt/ext/projects/pasteur/src/pasteur/hierarchy.py
Function: find_smallest_group at line 133

Line #      Hits         Time  Per Hit   % Time  Line Contents
   133                                           def find_smallest_group(
   134                                               counts: np.ndarray, parent: list[N]
   135                                           ) -> tuple[list, set, set, float]:
   136                                               """Finds groups `a`, `b` which when combined form `c`, where `c` is the smallest
   137                                               group that can be formed by any two nodes in the tree, which are valid to merge.
   138                                           
   139                                               Returns the parent `node` of `a` and `b`, `a`, `b`, and the size of the resulting group.
   140                                           
   141                              